In [ ]:
import json
import re
import pandas as pd
from datetime import datetime

In [4]:
with open("Data/companyfacts/CIK0000320193.json") as f:
    data = json.load(f)

In [5]:
def fiscal_year_checker(end, start):
    if not start:
        return True

    return (
        datetime.strptime(end, "%Y-%m-%d") - datetime.strptime(start, "%Y-%m-%d")
    ).days > 300


def extract_us_gaap_data(facts, tag, max_years=10):
    results = []
    if "us-gaap" in facts and tag in facts["us-gaap"]:
        for unit_entries in facts["us-gaap"][tag].get("units", {}).values():
            for entry in unit_entries:
                # check if the entry is fiscal year-end and the difference between start and end date is a year
                if entry.get("fp") == "FY" and fiscal_year_checker(entry.get("end"), entry.get("start")):
                    try:
                        fy = datetime.strptime(entry["end"], "%Y-%m-%d").year
                        value = entry["val"]
                        if isinstance(value, (int, float)):
                            results.append((fy, value))
                    except:
                        continue

    results = list({y: v for y, v in sorted(results, reverse=True)}.items())
    return results[:max_years]


facts = data.get("facts", {})

net_income_10yr = extract_us_gaap_data(facts, "NetIncomeLoss", 10)
dividends_20yr = extract_us_gaap_data(facts, "CommonStockDividendsPerShareDeclared", 20)
eps_10yr = extract_us_gaap_data(facts, "EarningsPerShareDiluted", 10)
book_value_3yr = extract_us_gaap_data(facts, "BookValuePerShare", 3)

In [90]:
net_income_10yr

[(2024, 93736000000),
 (2023, 96995000000),
 (2022, 99803000000),
 (2021, 94680000000),
 (2020, 57411000000),
 (2019, 55256000000),
 (2018, 59531000000),
 (2017, 48351000000),
 (2016, 45687000000),
 (2015, 53394000000)]

In [91]:
dividends_20yr

[(2024, 0.98),
 (2023, 0.94),
 (2022, 0.9),
 (2021, 0.85),
 (2020, 0.795),
 (2019, 0.75),
 (2018, 0.68),
 (2017, 2.4),
 (2016, 2.18),
 (2015, 1.98),
 (2014, 1.82),
 (2013, 1.64),
 (2012, 0.38),
 (2011, 0),
 (2010, 0)]

In [8]:
eps_10yr

[(2024, 6.08),
 (2023, 6.13),
 (2022, 6.11),
 (2021, 5.61),
 (2020, 3.28),
 (2019, 2.97),
 (2018, 2.98),
 (2017, 9.21),
 (2016, 8.31),
 (2015, 9.22)]

In [95]:
eps_10yr[:3]

[(2024, 6.08), (2023, 6.13), (2022, 6.11)]

In [94]:
book_value_3yr

[]

In [104]:
assets_10yr = extract_us_gaap_data(facts, "Assets", 10)

In [105]:
assets_10yr

[(2024, 364980000000),
 (2023, 352583000000),
 (2022, 352755000000),
 (2021, 351002000000),
 (2020, 323888000000),
 (2019, 338516000000),
 (2018, 365725000000),
 (2017, 375319000000),
 (2016, 321686000000),
 (2015, 290345000000)]

In [106]:
liabilities_10yr = extract_us_gaap_data(facts, "Liabilities", 10)

In [107]:
liabilities_10yr

[(2024, 308030000000),
 (2023, 290437000000),
 (2022, 302083000000),
 (2021, 287912000000),
 (2020, 258549000000),
 (2019, 248028000000),
 (2018, 258578000000),
 (2017, 241272000000),
 (2016, 193437000000),
 (2015, 170990000000)]

In [24]:
stockholder_equity_10yr = extract_us_gaap_data(facts, "StockholdersEquity", 10)
stockholder_equity_10yr

[]

In [127]:
pd.DataFrame(assets_10yr, columns=["Year", "Assets"]).iloc[:, 1]

0    364980000000
1    352583000000
2    352755000000
3    351002000000
4    323888000000
5    338516000000
6    365725000000
7    375319000000
8    321686000000
9    290345000000
Name: Assets, dtype: int64

In [128]:
pd.DataFrame(liabilities_10yr, columns=["Year", "Liabilities"]).iloc[:, 1]

0    308030000000
1    290437000000
2    302083000000
3    287912000000
4    258549000000
5    248028000000
6    258578000000
7    241272000000
8    193437000000
9    170990000000
Name: Liabilities, dtype: int64

In [132]:
(
    pd.DataFrame(assets_10yr, columns=["Year", "Assets"]).iloc[:, 1]
    - pd.DataFrame(liabilities_10yr, columns=["Year", "Liabilities"]).iloc[:, 1]
) / pd.DataFrame(shares_10yr, columns=["Year", "Shares"]).iloc[:, 1]

0     3.767335
1     3.996512
2     3.178238
3     3.840678
4     3.848731
5    20.365337
6    22.533610
7    26.149384
8    24.033922
9    21.394566
dtype: float64

In [131]:
pd.DataFrame(shares_10yr, columns=["Year", "Shares"]).iloc[:, 1]

0    15116786000
1    15550061000
2    15943425000
3    16426786000
4    16976763000
5     4443236000
6     4754986000
7     5126201000
8     5336166000
9     5578753000
Name: Shares, dtype: int64

In [108]:
shares_10yr = extract_us_gaap_data(facts, "CommonStockSharesOutstanding", 10)

In [109]:
shares_10yr

[(2024, 15116786000),
 (2023, 15550061000),
 (2022, 15943425000),
 (2021, 16426786000),
 (2020, 16976763000),
 (2019, 4443236000),
 (2018, 4754986000),
 (2017, 5126201000),
 (2016, 5336166000),
 (2015, 5578753000)]

In [7]:
shares_10yr = extract_us_gaap_data(
    facts, "WeightedAverageNumberOfDilutedSharesOutstanding", 50
)
shares_10yr

[(2024, 15408095000),
 (2023, 15812547000),
 (2022, 16325819000),
 (2021, 16864919000),
 (2020, 17528214000),
 (2019, 4648913000),
 (2018, 5000109000),
 (2017, 5251692000),
 (2016, 5500281000),
 (2015, 5793069000),
 (2014, 6122663000),
 (2013, 931662000),
 (2012, 945355000),
 (2011, 936645000),
 (2010, 924712000),
 (2009, 907005000),
 (2008, 902139000),
 (2007, 889292000)]

In [32]:
shares_10yr = extract_us_gaap_data(
    facts, "WeightedAverageNumberOfDilutedSharesOutstanding", 50
)
shares_10yr

[(2024, 15408095000),
 (2023, 15812547000),
 (2022, 16325819000),
 (2021, 16864919000),
 (2020, 17528214000),
 (2019, 18595651000),
 (2018, 20000435000),
 (2017, 5251692000),
 (2016, 5500281000),
 (2015, 5793069000),
 (2014, 6122663000),
 (2013, 6521634000),
 (2011, 936645000),
 (2010, 924712000),
 (2009, 907005000),
 (2008, 902139000),
 (2007, 889292000)]

In [31]:
shares_10yr = extract_us_gaap_data(facts, "CommonStockSharesIssued", 50)
shares_10yr

[]

In [115]:
sales_10yr = extract_us_gaap_data(
    facts, "RevenueFromContractWithCustomerExcludingAssessedTax", 10
)

In [116]:
sales_10yr

[(2024, 391035000000),
 (2023, 383285000000),
 (2022, 394328000000),
 (2021, 365817000000),
 (2020, 274515000000),
 (2019, 260174000000),
 (2018, 265595000000),
 (2017, 229234000000)]

In [9]:
eps_10yr

[(2024, 6.08),
 (2023, 6.13),
 (2022, 6.11),
 (2021, 5.61),
 (2020, 3.28),
 (2019, 2.97),
 (2018, 2.98),
 (2017, 9.21),
 (2016, 8.31),
 (2015, 9.22)]

In [34]:
stock_split_ratio_10yrs = extract_us_gaap_data(
    facts, "StockholdersEquityNoteStockSplitConversionRatio1", 10
)
stock_split_ratio_10yrs

[]

In [21]:
def check_annual_report(s):
    if not s:
        return False
    return bool(re.match(r"^CY\d{4}$", s))

def extract_us_gaap_data(facts, tag, max_years=10):
    results = []
    if "us-gaap" in facts and tag in facts["us-gaap"]:
        for unit_entries in facts["us-gaap"][tag].get("units", {}).values():
            for entry in unit_entries:
                # check if the entry is fiscal year-end and the difference between start and end date is a year
                if entry.get("fp") == "FY" and check_annual_report(entry.get("frame")):
                    try:
                        fy = datetime.strptime(entry["end"], "%Y-%m-%d").year
                        value = entry["val"]
                        if isinstance(value, (int, float)):
                            results.append((fy, value))
                    except:
                        continue

    results = list({y: v for y, v in sorted(results, reverse=True)}.items())
    return results[:max_years]


facts = data.get("facts", {})

net_income_10yr = extract_us_gaap_data(facts, "NetIncomeLoss", 10)
dividends_20yr = extract_us_gaap_data(facts, "CommonStockDividendsPerShareDeclared", 20)
eps_10yr = extract_us_gaap_data(facts, "EarningsPerShareDiluted", 10)
book_value_3yr = extract_us_gaap_data(facts, "BookValuePerShare", 3)